<a href="https://colab.research.google.com/github/minhajuddinali/Aves-detection/blob/main/birds_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Conv2D,Add,MaxPooling2D, Dense, BatchNormalization,Input,Flatten, Dropout,GlobalMaxPooling2D,Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle


In [ ]:
!kaggle datasets download gpiosenka/100-bird-species

100% 1.69G/1.69G [01:28<00:00, 22.0MB/s]
100% 1.69G/1.69G [01:28<00:00, 20.4MB/s]


In [ ]:
 !unzip 100-bird-species.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/VIOLET TURACO/129.jpg  
  inflating: train/VIOLET TURACO/130.jpg  
  inflating: train/VIOLET TURACO/131.jpg  
  inflating: train/VIOLET TURACO/132.jpg  
  inflating: train/VIOLET TURACO/133.jpg  
  inflating: train/VIOLET TURACO/134.jpg  
  inflating: train/VIOLET TURACO/135.jpg  
  inflating: train/VIOLET TURACO/136.jpg  
  inflating: train/VIOLET TURACO/137.jpg  
  inflating: train/VIOLET TURACO/138.jpg  
  inflating: train/VIOLET TURACO/139.jpg  
  inflating: train/VIOLET TURACO/140.jpg  
  inflating: train/VIOLET TURACO/141.jpg  
  inflating: train/VIOLET TURACO/142.jpg  
  inflating: train/VIOLET TURACO/143.jpg  
  inflating: train/VIOLET TURACO/144.jpg  
  inflating: train/VIOLET TURACO/145.jpg  
  inflating: train/VIOLET TURACO/146.jpg  
  inflating: train/VIOLET TURACO/147.jpg  
  inflating: train/VIOLET TURACO/148.jpg  
  inflating: train/VIOLET TURACO/149.jpg  
  inflating: train/VIOLET TURACO/150.jpg  
  i

In [ ]:
train = '/content/train'
validation = '/content/valid'
test = '/content/test'

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)#initialize train generator 
                                 


valid_datagen = ImageDataGenerator(rescale = 1.0/255.) #initialize validation generator 

test_datagen = ImageDataGenerator(rescale = 1.0/255.) #initialize validation generator 

In [ ]:
train_generator = train_datagen.flow_from_directory(train, target_size=(256,256),batch_size=32,class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(validation, target_size=(256,256),batch_size=32,class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test, target_size=(256,256),batch_size=32,class_mode='categorical')

Found 70626 images belonging to 450 classes.
Found 2250 images belonging to 450 classes.
Found 2250 images belonging to 450 classes.


In [ ]:
#Load Model
inception = tf.keras.applications.InceptionV3(weights='imagenet',include_top=False,input_shape=(256,256,3))


87910968/87910968 [==============================] - 5s 0us/step


In [ ]:
inception.trainable = True
for layer in inception.layers[:197]:
    layer.trainable = False 

In [ ]:
for idx, layer in enumerate(inception.layers):
    print(f' {idx}:  {layer.name}: trainable = {layer.trainable}')


 0:  input_2: trainable = False
 1:  conv2d_94: trainable = False
 2:  batch_normalization_94: trainable = False
 3:  activation_94: trainable = False
 4:  conv2d_95: trainable = False
 5:  batch_normalization_95: trainable = False
 6:  activation_95: trainable = False
 7:  conv2d_96: trainable = False
 8:  batch_normalization_96: trainable = False
 9:  activation_96: trainable = False
 10:  max_pooling2d_4: trainable = False
 11:  conv2d_97: trainable = False
 12:  batch_normalization_97: trainable = False
 13:  activation_97: trainable = False
 14:  conv2d_98: trainable = False
 15:  batch_normalization_98: trainable = False
 16:  activation_98: trainable = False
 17:  max_pooling2d_5: trainable = False
 18:  conv2d_102: trainable = False
 19:  batch_normalization_102: trainable = False
 20:  activation_102: trainable = False
 21:  conv2d_100: trainable = False
 22:  conv2d_103: trainable = False
 23:  batch_normalization_100: trainable = False
 24:  batch_normalization_103: trainabl

In [ ]:
#get the last layer    
last_layer = inception.get_layer('mixed7')

print('last layer output shape: ', last_layer.output_shape)

layer_output = last_layer.output

last layer output shape:  (None, 14, 14, 768)


In [ ]:
n_categories = len(os.listdir('/content/train'))# number of categories
print(n_categories)

450


In [ ]:
# x  = BatchNormalization()(layer_output)
x = Flatten()(layer_output)
x = Dense(1024,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = Dropout(0.4)(x)
x = Dense(n_categories, activation='softmax')(x)

model = Model(inputs=inception.inputs, outputs=x)


In [ ]:
# Set the training parameters
model.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

In [ ]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
    
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
history = mynewmodel.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 25,
callbacks=[callback])

Epoch 1/25
2208/2208 [==============================] - 435s 195ms/step - loss: 0.2805 - accuracy: 0.9658 - val_loss: 0.4010 - val_accuracy: 0.9413 - lr: 6.7032e-05
Epoch 2/25
2208/2208 [==============================] - 429s 194ms/step - loss: 0.2689 - accuracy: 0.9668 - val_loss: 0.3833 - val_accuracy: 0.9449 - lr: 6.7032e-05
Epoch 3/25
2208/2208 [==============================] - 431s 195ms/step - loss: 0.2584 - accuracy: 0.9688 - val_loss: 0.3844 - val_accuracy: 0.9431 - lr: 6.7032e-05
Epoch 4/25
2208/2208 [==============================] - 430s 195ms/step - loss: 0.2513 - accuracy: 0.9685 - val_loss: 0.4044 - val_accuracy: 0.9458 - lr: 6.7032e-05
Epoch 5/25
2208/2208 [==============================] - 431s 195ms/step - loss: 0.2464 - accuracy: 0.9689 - val_loss: 0.3865 - val_accuracy: 0.9493 - lr: 6.7032e-05
Epoch 6/25
2208/2208 [==============================] - 431s 195ms/step - loss: 0.2377 - accuracy: 0.9699 - val_loss: 0.3799 - val_accuracy: 0.9444 - lr: 6.7032e-05
Epoch 7/25

In [ ]:
mynewmodel.save('/content/drive/MyDrive/Colab Notebooks/models/ekbaarphirmodel.h5')

In [ ]:
mynewmodel.evaluate(test_generator)

71/71 [==============================] - 10s 140ms/step - loss: 0.1971 - accuracy: 0.9671


[0.19705578684806824, 0.9671111106872559]

In [ ]:
from tensorflow import keras
mynewmodel = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/models/probablyfinalmodel.h5')